<a href="https://colab.research.google.com/github/juansraigosoi-ecci/Regresion_Lineal./blob/main/Regresion_Lineal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Regresion lineal**

**integrantes**
1. Juan Sebastian Raigoso Ibañez
2. Laura Tatiana Gonzalez Garcia
3. Rubiel Santiago Garcia Garzon

**Planteamiento del problema**

**Objetivo:**  
Predecir la demanda semanal de un SKU para optimizar el reabastecimiento.

**Variable dependiente (y):**  
- `demanda_semanal` (unidades vendidas por semana).

**Variables independientes (X):**  
- `precio_cop`: precio unitario en COP.  
- `promocion`: indicador (0 = no hay promo, 1 = sí).  
- `stock_inicio`: inventario disponible al inicio de la semana.  
- `lead_time_dias`: tiempo de reposición del proveedor (días).  
- `indice_estacional`: factor de estacionalidad (0.5 baja – 1.5 alta).

> Tipo de modelo: **Regresión lineal múltiple**.


**Parte 2: Preprocesamiento de Datos y *Leakage***
Basado en el caso que definiste en la Parte 1:

Lista entre 3 y 5 transformaciones que aplicarías a tus datos (ej. imputación de valores faltantes, codificación de variables categóricas, escalado, creación de lags, etc.) y justifica por qué cada una es necesaria.
Señala un posible riesgo de data leakage (fuga de datos) en tu plan y explica cómo lo evitarías usando un pipeline de preprocesamiento.

In [ ]:
import pandas as pd
import numpy as np

np.random.seed(42)   # reproducibilidad

n = 100
precio_cop = np.random.uniform(35000, 50000, size=n)
promocion = np.random.choice([0,1], size=n, p=[0.6,0.4])
stock_inicio = np.random.uniform(50,200, size=n)
lead_time_dias = np.random.randint(5,11, size=n)
indice_estacional = np.random.uniform(0.8,1.5, size=n)

# Fórmula de demanda con ruido
demanda_semanal = (
    -0.004 * precio_cop +
    50 * promocion +
    0.25 * stock_inicio -
    3 * lead_time_dias +
    80 * indice_estacional +
    np.random.normal(scale=10, size=n)
).round().astype(int)

df = pd.DataFrame({
    "precio_cop": precio_cop,
    "promocion": promocion,
    "stock_inicio": stock_inicio,
    "lead_time_dias": lead_time_dias,
    "indice_estacional": indice_estacional,
    "demanda_semanal": demanda_semanal
})

df.head()


**Parte 3: Interpretación y Métricas de Regresión Simple**
Para esta sección, elige un caso simple de regresión (puede ser el tuyo o uno hipotético, como predecir la demanda de un producto según su precio).

Define claramente las variables
 y
 junto con sus unidades (ej.
: número de unidades vendidas,
: precio en dólares).
Supón que entrenas un modelo y obtienes una pendiente de
. Escribe una interpretación clara y concisa de este coeficiente en el contexto de tu problema.
¿Qué métrica de evaluación usarías (MAE, RMSE, o MAPE) y por qué es la más adecuada para tu caso?
Menciona un supuesto del modelo de regresión lineal que validarías (ej. linealidad, homocedasticidad) y explica cómo lo harías (usando un gráfico o una prueba estadística).

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.pairplot(df, diag_kind="kde")
plt.show()

df.describe()


**Parte 4: Regresión Múltiple y Colinealidad**
Volviendo a tu caso de la Parte 1 (con múltiples variables).

Escribe el vector de variables
 y la respuesta
.
Explica cómo interpretarías el coeficiente de una de tus variables clave (incluyendo unidades y el sentido de la relación: positiva o negativa).
Si sospecharas que existe colinealidad entre tus variables, menciona dos acciones que podrías tomar para mitigarla.

In [ ]:
import statsmodels.api as sm

y = df["demanda_semanal"]
X = df[["precio_cop","promocion","stock_inicio","lead_time_dias","indice_estacional"]]
X = sm.add_constant(X)

modelo = sm.OLS(y, X).fit()
print(modelo.summary())


Interpretación esperada (aprox):

R² alto (>0.9).

precio_cop y lead_time_dias con signo negativo.

promocion, stock_inicio e indice_estacional con signo positivo.

**Parte 5: Interacciones y Multicolinealidad (VIF)**
Plantea un caso con una variable
 y entre 4 y 6 variables
. ¿Qué término de interacción entre dos variables podrías añadir al modelo y por qué crees que sería útil?
Si al calcular el Factor de Inflación de la Varianza (VIF) para una variable, obtienes un valor alto (ej. > 10), menciona dos acciones que podrías tomar para solucionarlo.

In [ ]:
from scipy.stats import shapiro
import statsmodels.stats.api as sms
from statsmodels.stats.stattools import durbin_watson
from statsmodels.stats.outliers_influence import variance_inflation_factor

resid = modelo.resid
fitted = modelo.fittedvalues

# 1) Normalidad de residuos
sns.histplot(resid, kde=True)
plt.title("Distribución de los residuos")
plt.show()

print("Shapiro-Wilk:", shapiro(resid))

# 2) Homocedasticidad (Breusch-Pagan)
bp = sms.het_breuschpagan(resid, modelo.model.exog)
print("Breusch-Pagan (LM, p-valor):", bp[0], bp[1])

# 3) Independencia (Durbin-Watson)
print("Durbin-Watson:", durbin_watson(resid))

# 4) Colinealidad (VIF)
X_vif = X.drop(columns=["const"])
vif = pd.DataFrame()
vif["Variable"] = X_vif.columns
vif["VIF"] = [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]
vif


**Parte 6: Variables Categóricas e Interacciones**
Define una variable categórica para tu caso (puedes inventarla si no la tenías). Elige una de sus categorías como el nivel base o de referencia y justifica tu elección.
Crea una interacción entre una variable numérica y la variable categórica que definiste. Explica cómo se interpretaría el coeficiente de esta interacción.

- **Precio (coeficiente negativo):** al aumentar 1 COP, la demanda baja aprox 0.004 uds (≈4 uds cada mil COP).
- **Promoción:** incrementa ventas en ~50 unidades.
- **Stock al inicio:** cada unidad adicional disponible suma ~0.25 uds vendidas.
- **Lead time:** cada día extra de reposición reduce demanda en ~3 uds (posibles quiebres).
- **Estacionalidad:** subir un punto (ej. de 1.0 a 2.0) aumenta ventas en ~80 uds.

> Todas las variables tienen p-valor < 0.05 → son significativas.


**Parte 7: Conceptos Clave de Clasificación**
Aunque el taller se centra en regresión, estos conceptos son fundamentales en Machine Learning.

Explica qué es la curva ROC y para qué se utiliza en un problema de clasificación.
Define el concepto de accuracy (exactitud) y menciona una situación en la que podría ser una métrica engañosa.
Describe qué es una matriz de confusión y cómo se interpretan sus componentes (Verdaderos Positivos, Falsos Positivos, Verdaderos Negativos, Falsos Negativos).

- El modelo explica bien la demanda (R² alto).  
- Las promociones y estacionalidad impulsan ventas; el precio y lead time las reducen.  
- Mantener inventarios suficientes y acortar el lead time ayuda a no perder ventas.
- Puede ampliarse el modelo con datos reales, variables de marketing o clima.
